In [ ]:
#importing all libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
%matplotlib inline  
# style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import os
import random
from random import shuffle

from tqdm import tqdm
import cv2

from keras.utils import to_categorical
from keras.models import Sequential,load_model,model_from_json
from keras.layers import Dense
from keras.optimizers import RMSprop,Adadelta,Adam,Adamax,SGD
from keras import backend as K
from keras.layers import Dropout,Flatten,AveragePooling2D,MaxPooling2D,Conv2D,Activation,BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
#setting path to dataset 
curr_path = os.getcwd() + '/drive/My Drive/datasets'
data_path = curr_path+'/Large'

In [ ]:
#mapping of X and Y label matlab img and flower name
# imglist = list()


# for dr in os.listdir(data_path) :
#     if dr not in ['Traffic Light','Palm','Other','Mountain','Motorcycle','Hydrant','Crosswalk','Chimney','Car','Bus','Bridge','Bicycle'] :
#         continue
#     img_path = os.path.join(data_path,dr)
#     print(dr)
#     for img in tqdm(os.listdir(img_path) ):
#         path = os.path.join(img_path,img)
#         pic = cv2.imread(path)
#         pic = cv2.resize(pic,(96,96))
#         imglist.append([pic,dr])


imglist = list()


for dr in os.listdir(data_path) :
    if dr not in ['Traffic Light','Palm','Hydrant','Crosswalk','Car','Bus','Bicycle'] :
        continue
    img_path = os.path.join(data_path,dr)
    print(dr)
    for img in tqdm(os.listdir(img_path) ):
        path = os.path.join(img_path,img)
        pic = cv2.imread(path)
        pic = cv2.resize(pic,(96,96))
        
        imglist.append([pic,dr])

In [ ]:
X,y_temp = map(list,zip(*imglist))

In [ ]:
# just checking if x and y are mapped correctly
fig,ax=plt.subplots(5,2)
fig.set_size_inches(15,15)
for i in range(5):
    for j in range (2):
        l=random.randint(0,len(y_temp))
        ax[i,j].imshow(X[l])
        ax[i,j].set_title('Caption: '+y_temp[l])
        
plt.tight_layout()

In [ ]:
  #le -> performing one hot encoding( # [1,0,0,0,0], [0,1,0,0,0])
  le = LabelEncoder()
  Y = le.fit_transform(y_temp)
  Y = to_categorical(Y,12)

  # normalization of the X-label between 0 and 1
  X=np.array(X)
  X=X/255

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.2,random_state=42)

In [ ]:
#making cnn model

model = Sequential()
model.add(Conv2D(filters=32,kernel_size=(5,5),input_shape=(96,96,3),padding='Same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,kernel_size=(3,3),padding='Same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=96,kernel_size=(2,2),padding='Same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(12,activation='softmax'))

In [ ]:
# setting batch_size nd epooch for training the model
batch_size = 128
epooch = 50

# red_lr = ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.1)

In [ ]:
# data augmentation to prevent overfitting
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

In [ ]:
model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# training of the model
import time
start = time.perf_counter()
History = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epooch, validation_data = (x_test,y_test),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size)
end = time.perf_counter()
# hist = model.fit(x=x_train,y=y_train,batch_size=batch_size,epochs=epooch,callbacks=red_lr,validation_data=(x_test,y_test),steps_per_epoch=x_train.shape[0] // batch_size)

In [ ]:
print("Time taken: ",end-start)

In [ ]:
# saving the model
model.save_weights(os.path.join(curr_path ,'cnn_base.h5'))

with open(os.path.join(curr_path,"cnn_base.json"), "w") as json_file:
    json_file.write(model.to_json())

In [ ]:
#accuracy graph of our model
plt.plot(History.history['accuracy'])
plt.plot(History.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
#import our model from json
with open(os.path.join(curr_path,'cnn_base.json'), 'r') as f:
    loaded_model_json = f.read()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights(os.path.join(curr_path,"cnn_base.h5"))

In [ ]:
# some debugging
pred = loaded_model.predict(x_test)
pred_final = np.argmax(pred,axis=1)
pred_final=pred_final.reshape(1917,1)
pos = 0
for i in range(1917):
  if y_test[i][pred_final[i]] == 1:
    pos += 1
accuracy = pos/len(y_test)
accuracy*100

In [ ]:
# print(le.inverse_transform(y_test[0]),pred_final[0])
d = {0:'Bicycle',1:'Bus',2:'Car',3:'Crosswalk',4:'Hydrant',5:'Palm',6:'Traffic Light'}
print(str(le.inverse_transform([pred_final[200]])))
print(str([pred_final[200]]))
print(np.argmax([y_test[200]]))
print(d[np.argmax([y_test[200]])])
plt.imshow(x_test[200])

In [ ]:
# checking our result. Comparing some of the predicted and test values
count=0
fig,ax=plt.subplots(8,2)
fig.set_size_inches(15,15)
for i in range (8):
    for j in range (2):
        ax[i,j].imshow(x_test[count])
        ax[i,j].set_title("Prediction : "+str(le.inverse_transform([pred_final[count]]))+"\n"+"Actual : "+d[np.argmax([y_test[count]])])
        plt.tight_layout()
        count+=2